In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
import pickle

path = os.getcwd().split(os.sep + 'GUI')[0]
if path not in sys.path:
    print("not here")
    sys.path.append(path)

from neurolib.models.aln import ALNModel
from neurolib.utils import plotFunctions as plotFunc
from neurolib.utils import costFunctions as cost
import neurolib.dashboard.functions as functions
import neurolib.dashboard.data as data
    
# This will reload all imports as soon as the code changes
%load_ext autoreload
%autoreload 2 

#path = os.path.join(os.getcwd(), "plots")

not here


### Bistability

In [2]:
aln = ALNModel()
N = aln.params.N

data.set_parameters(aln)

state_vars = aln.state_vars
init_vars = aln.init_vars

##############################################################
def setinit(init_vars_, model):
    state_vars = model.state_vars
    init_vars = model.init_vars
    for iv in range(len(init_vars)):
        for sv in range(len(state_vars)):
            if state_vars[sv] in init_vars[iv]:
                #print("set init vars ", )
                if model.params[init_vars[iv]].ndim == 2:
                    model.params[init_vars[iv]][0,:] = init_vars_[sv]
                else:
                    model.params[init_vars[iv]][0] = init_vars_[sv]
                    
##############################################################               
def setmaxmincontrol(max_c_c, min_c_c, max_c_r, min_c_r):
    import numpy as np
    
    max_cntrl = np.zeros(( 6 ))
    min_cntrl = np.zeros(( 6 ))
    
    max_cntrl[0] = max_c_c
    min_cntrl[0] = min_c_c
    max_cntrl[1] = max_c_c
    min_cntrl[1] = min_c_c
    max_cntrl[2] = max_c_r
    min_cntrl[2] = min_c_r
    max_cntrl[3] = max_c_r
    min_cntrl[3] = min_c_r
    max_cntrl[4] = max_c_r
    min_cntrl[4] = min_c_r
    max_cntrl[5] = max_c_r
    min_cntrl[5] = min_c_r
            
    return max_cntrl, min_cntrl

In [3]:
##### LOAD BOUNDARIES
data_file = 'bi.pickle'
with open(data_file,'rb') as f:
    load_array= pickle.load(f)
exc = load_array[0]
inh = load_array[1]
print(len(exc))

147


In [4]:
t0_array = np.arange(380., -1, -1.)
t_pen = t0_array / 400.

bestControl_1 = [None] * len(t_pen)
bestState_1 = [None] * len(t_pen)
cost_1 = [None] * len(t_pen)
runtime_1 = [None] * len(t_pen)
grad_1 = [None] * len(t_pen)
phi_1 = [None] * len(t_pen)
costnode_1 = [None] * len(t_pen)
weights_1 = [None] * len(t_pen)

In [5]:
initVars = [None] * len(exc)
target = [None] * len(exc)
cost_uncontrolled = [None] * len(exc)

cgv_list = [None, "HS", "FR", "PR", "CD", "LS", "DY", "WYL", "HZ", None]

In [6]:
dur_pre = 10
dur_post = 10

i = 20

n_pre = int(np.around(dur_pre/aln.params.dt + 1.,1))
n_post = int(np.around(dur_post/aln.params.dt + 1.,1))

tol = 1e-32
start_step = 10.
c_scheme = np.zeros(( 1,1 ))
c_scheme[0,0] = 1.
u_mat = np.identity(1)
u_scheme = np.array([[1.]])

c_var = [ [0], [1], [0,1]]
p_var = [ [0], [0], [0]]

### CURRENTS
cntrl_vars_0 = [0,1]
prec_vars = [0]

max_I = [-3., 3.]
factor_wp = 1. / 0.05
factor_ws = 1.
factor_we = 0.
dur = 400
trans_time = 0.95
    
maxC = [5., -5., 0.18, 0.]

n_dur = int(np.around(dur/aln.params.dt + 1.,1))
max_cntrl, min_cntrl = setmaxmincontrol(maxC[0], maxC[1], maxC[2], maxC[3])

In [7]:
init_file = 'control_3.pickle'
final_file_1 = 'control_tradeoff_3a.pickle'

In [8]:
if os.path.isfile(init_file) :
    print("init file found")
    
    with open(init_file,'rb') as f:
        load_array = pickle.load(f)

    print(len(load_array[0]))

    bestControl_1[0] = load_array[0][i]
    bestState_1[0] = load_array[1][i]
    cost_1 [0]= load_array[2][i]
    runtime_1[0] = load_array[3][i]
    grad_1 [0]= load_array[4][i]
    phi_1[0] = load_array[5][i]
    costnode_1[0] = load_array[6][i]
    weights_1[0] = load_array[7][i]

init file found
147


In [9]:
if os.path.isfile(final_file_1) :
    print("final file found")
    
    with open(final_file_1,'rb') as f:
        load_array = pickle.load(f)

    print(len(load_array[0]))

    bestControl_1 = load_array[0]
    bestState_1 = load_array[1]
    cost_1 = load_array[2]
    runtime_1 = load_array[3]
    grad_1 = load_array[4]
    phi_1 = load_array[5]
    costnode_1 = load_array[6]
    weights_1 = load_array[7]

In [10]:
# get initial parameters and target states

print("------- ", i, exc[i], inh[i])
aln.params.ext_exc_current = exc[i] * 5.
aln.params.ext_inh_current = inh[i] * 5.

aln.params.duration = 3000.

control0 = aln.getZeroControl()
control0 = functions.step_control(aln, maxI_ = max_I[0])

aln.run(control=control0)

target_rates = np.zeros((2))
target_rates[0] = aln.rates_exc[0,-1] 
target_rates[1] = aln.rates_inh[0,-1]

control0 = functions.step_control(aln, maxI_ = max_I[1])
aln.run(control=control0)

init_state_vars = np.zeros(( len(state_vars) ))
for j in range(len(state_vars)):
    if aln.state[state_vars[j]].size == 1:
        init_state_vars[j] = aln.state[state_vars[j]][0]
    else:
        init_state_vars[j] = aln.state[state_vars[j]][0,-1]

initVars[i] = init_state_vars

aln.params.duration = dur

target[i] = aln.getZeroTarget()
target[i][:,0,:] = target_rates[0]
target[i][:,1,:] = target_rates[1]

-------  20 0.4500000000000001 0.4750000000000002


In [11]:
# get uncontrolled cost

data.set_parameters(aln)
aln.params.ext_exc_current = exc[i] * 5.
aln.params.ext_inh_current = inh[i] * 5.
aln.params.duration = dur

cgv = None

In [12]:
print(np.argmax(np.abs(bestControl_1[0]), axis=2))
print(bestControl_1[0][0,0,1650:1700]/5.)
print(bestControl_1[0][0,0,1674]/5.)

max_percent = (np.argmax(np.abs(bestControl_1[0]), axis=2)[0,0] -100) / 3800
print(max_percent)

[[1610    0    0    0    0    0]]
[-0.09724783 -0.09678153 -0.0963073  -0.0958246  -0.09533229 -0.09483421
 -0.09432871 -0.09387099 -0.09338664 -0.09289497 -0.092398   -0.0918953
 -0.091388   -0.09088345 -0.09037117 -0.08985366 -0.08933102 -0.08880229
 -0.08826829 -0.08772984 -0.0871863  -0.08669257 -0.08617881 -0.08566055
 -0.08514039 -0.08461651 -0.08408871 -0.08355443 -0.08301978 -0.08248201
 -0.08195265 -0.0814186  -0.08088168 -0.08034171 -0.07979948 -0.07925444
 -0.07870699 -0.07815736 -0.0776487  -0.07713796 -0.07662428 -0.07610876
 -0.07559157 -0.07506964 -0.07454789 -0.07402488 -0.07350076 -0.07297504
 -0.07244951 -0.07191844]
-0.08514038855988344
0.3973684210526316


In [13]:
factor_iteration = 1.

##### initial guess
weight_ = 1.
cost.setParams(weight_ * factor_wp, weight_ * factor_we, weight_ * factor_ws)

setinit(initVars[i], aln)

weights_1[0] = cost.getParams()

for j in range(1, 381):

    dur = 400. - np.around(j/10., 1)
    T = 4001 - j
    aln.params.duration = dur

    factor_wp = dur / 20.
    cost.setParams(weight_ * factor_wp, weight_ * factor_we, weight_ * factor_ws)

    print('-----', j)
    setinit(initVars[i], aln)

    max_it = 3.
    if j > 170:
        max_it = 40.
    elif j > 140:
        max_it = 30.
    elif j > 100.:
        max_it = 20.
    
    if type(bestControl_1[j]) == type(None):
        total_shift = round(j * max_percent * 10)
        prev_shift = round( (j-1) * max_percent * 10)
        shift_ind = total_shift - prev_shift
        #print(total_shift, prev_shift, shift_ind)
        control0 = np.zeros(( bestControl_1[j-1][:,:,100:-101].shape ))
        control0[:,:,:-shift_ind] = 1. * bestControl_1[j-1][:,:,100+shift_ind:-101]
    else:
        if bestControl_1[j].shape[2] == T+200:
            control0 = bestControl_1[j][:,:,100:-100]
        else:
            control0 = bestControl_1[j][:,:,100:-(100+j)]

    bestControl_1[j], bestState_1[j], cost_1[j], runtime_1[j], grad_1[j], phi_1[j], costnode_1[j] = aln.A1(
        control0, target[i], c_scheme, u_mat, u_scheme, max_iteration_ = max_it, tolerance_ = tol,
        startStep_ = start_step, max_control_ = max_cntrl, min_control_ = min_cntrl, t_sim_ = dur,
        t_sim_pre_ = dur_pre, t_sim_post_ = dur_post, CGVar = cgv, control_variables_ = [0,1],
        prec_variables_ = prec_vars, transition_time_ = t_pen[j])

    print(costnode_1[j][0][0][0], costnode_1[j][2][0][1])

    print(costnode_1[0][0][0][0], costnode_1[0][2][0][1])
        
    with open(final_file_1,'wb') as f:
        pickle.dump([bestControl_1, bestState_1, cost_1, runtime_1, grad_1, phi_1,
                costnode_1, weights_1], f)

set cost params:  20.0 0.0 1.0
set cost params:  19.994999999999997 0.0 1.0
----- 1
interpolate adjoint :  True True True
RUN  0 , total integrated cost =  2.1464002606611516
Gradient descend method:  None
RUN  1 , total integrated cost =  2.1463976981356114
RUN  2 , total integrated cost =  2.146397696874784
RUN  3 , total integrated cost =  2.146397696801372
RUN  3 , total integrated cost =  2.146397696801372
Improved over  3  iterations in  26.416307265000796  seconds by  0.0001194492856910756  percent.


ERROR:root:Problem in initial value trasfer


Problem in initial value trasfer:  Vmean_exc -70.82163711951495 -70.82228271215305
3.529669528488474e-08 0.0
1.7978112780042006e-07 0.0
set cost params:  19.990000000000002 0.0 1.0
----- 2
interpolate adjoint :  True True True
RUN  0 , total integrated cost =  2.1463977559203826
Gradient descend method:  None
RUN  1 , total integrated cost =  2.146397755202459
RUN  2 , total integrated cost =  2.1463977549835493


ERROR:root:Problem in initial value trasfer


RUN  3 , total integrated cost =  2.1463977546491737
RUN  3 , total integrated cost =  2.1463977546491737
Improved over  3  iterations in  2.5384197029998177  seconds by  5.922522916534945e-08  percent.
Problem in initial value trasfer:  Vmean_exc -70.82507798928762 -70.82570735142966
3.71879095455505e-08 0.0
1.7978112780042006e-07 0.0
set cost params:  19.985 0.0 1.0
----- 3
interpolate adjoint :  True True True
RUN  0 , total integrated cost =  2.1463978263160715
Gradient descend method:  None
RUN  1 , total integrated cost =  2.1463978243103443
RUN  2 , total integrated cost =  2.146397823959515


...

RUN  40 , total integrated cost =  2.242655721363827
RUN  40 , total integrated cost =  2.242655721363827
Improved over  40  iterations in  21.35488799599989  seconds by  0.31386064748066644  percent.
Problem in initial value trasfer:  Vmean_exc -70.95859381489993 -70.95859382565803
0.0006086716890013472 0.0
1.7978112780042006e-07 0.0
set cost params:  18.43 0.0 1.0
----- 314
interpolate adjoint :  True True True
RUN  0 , total integrated cost =  2.254169940039866
Gradient descend method:  None
RUN  1 , total integrated cost =  2.2490592928997524
RUN  2 , total integrated cost =  2.2490478975425026
RUN  3 , total integrated cost =  2.2490294101310586
RUN  4 , total integrated cost =  2.2490156474472878
RUN  5 , total integrated cost =  2.248967831863396
RUN  6 , total integrated cost =  2.248925188446291
RUN  7 , total integrated cost =  2.248804365590699
RUN  8 , total integrated cost =  2.2486926034678607
RUN  9 , total integrated cost =  2.2486812533630594
RUN  10 , total integrated

ERROR:root:Problem in initial value trasfer


RUN  40 , total integrated cost =  2.2480039858527703
RUN  40 , total integrated cost =  2.2480039858527703
Improved over  40  iterations in  22.325802669998666  seconds by  0.27353546321297983  percent.
Problem in initial value trasfer:  Vmean_exc -70.95859388500128 -70.95859389542932
0.0006550818941589384 0.0
1.7978112780042006e-07 0.0
set cost params:  18.425 0.0 1.0
----- 315
interpolate adjoint :  True True True
RUN  0 , total integrated cost =  2.2622324150549935
Gradient descend method:  None
RUN  1 , total integrated cost =  2.255380408056152
RUN  2 , total integrated cost =  2.25536463973281
RUN  3 , total integrated cost =  2.2553444363325834
RUN  4 , total integrated cost =  2.2553325647925964
RUN  5 , total integrated cost =  2.2553122222720896
RUN  6 , total integrated cost =  2.255299381395261
RUN  7 , total integrated cost =  2.2552740638637845
RUN  8 , total integrated cost =  2.2552545378628666
RUN  9 , total integrated cost =  2.2551795131530885
RUN  10 , total integr

ERROR:root:Problem in initial value trasfer


Control only changes marginally.
RUN  34 , total integrated cost =  2.254239814446187
Improved over  34  iterations in  19.719174571997428  seconds by  0.35330590064999967  percent.
Problem in initial value trasfer:  Vmean_exc -70.95859397094986 -70.95859398097322
0.000683904514320137 0.0
1.7978112780042006e-07 0.0
set cost params:  18.419999999999998 0.0 1.0
----- 316
interpolate adjoint :  True True True
RUN  0 , total integrated cost =  2.2665790149801586
Gradient descend method:  None
RUN  1 , total integrated cost =  2.261209112203581
RUN  2 , total integrated cost =  2.261154284900059
RUN  3 , total integrated cost =  2.261108509007672
RUN  4 , total integrated cost =  2.260987521692146
RUN  5 , total integrated cost =  2.260906314107173
RUN  6 , total integrated cost =  2.2603245424271585
RUN  7 , total integrated cost =  2.260323910503857
RUN  8 , total integrated cost =  2.2603237047275284
RUN  9 , total integrated cost =  2.2603234563006342
RUN  10 , total integrated cost =  